<a id="title_ID"></a>
# JWST Data Products: Ensemble Processing Products
--------------------------------------------------------------
**Author**: Alicia Canipe (acanipe@stsci.edu) | **Latest update**: March 24, 2021.

## Table of contents
1. [Introduction](#intro)
   1. [Resources](#resources)   
2. [Data in MAST](#mast)
3. [Example data for this exercise](#example)
4. [Data products: stage 3 (combined, rectified exposures)](#stage3)
    1. [Imaging](#s3-imaging)
        1. [Input](#s3-imaging-input)
        2. [Output](#s3-imaging-output)
    2. [Spectroscopy](#s3-spectroscopy)
        1. [Input](#s3-spectroscopy-input)
        2. [Output](#s3-spectroscopy-output)
    3. [Aperture Masking Interferometry (AMI)](#s3-ami)
        1. [Input](#s3-ami-input)
        2. [Output](#s3-ami-output)
    4. [Coronagraphy](#s3-coronagraphy)
        1. [Input](#s3-coronagraphy-input)
        2. [Output](#s3-coronagraphy-output)
    5. [Time Series Observation (TSO)](#s3-tso)
        1. [Input](#s3-tso-input)
        2. [Output](#s3-tso-output)
5. [Examining the products](#examine)
    1. [Catalogs](#catalogs)
    2. [Combined image](#comb-image)
    3. [Combined spectrum](#comb-spec)
6. [The end](#bye-bye)
7. [Exercise](#exercise)

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

Welcome to the final module about JWST data products! In this module, we will learn more about the data products associated with Stage 3 in the JWST Calibration Pipeline (hereafter, the pipeline), the final stage of processing. You made it. 

### A.<font color='white'>-</font>Resources<a class="anchor" id="resources"></a>


Visit the [webpage for JWebbinars](https://www.stsci.edu/jwst/science-execution/jwebbinars) to find resources for:
* The Mikulski Archive for Space Telescopes (MAST) 
* JWST Documentation (JDox) for JWST data products
* The most up-to-date information about JWST data products in the pipeline readthedocs
* Pipeline roadmaps for when to recalibrate your data

Before we begin, import the libraries used in this notebook:

In [ ]:
# Module with functions to get information about objects:
import os

# To get data from Box
import requests

# Numpy library:
import numpy as np

# Scipy tools
from scipy.signal import medfilt

# Astropy tools:
from astropy.utils.data import download_file
from astropy.stats import sigma_clip
from astropy.table import Table
from astropy.io import fits, ascii

# The JWST models:
from jwst import datamodels

And set up matplotlib for plotting:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version if you want interactive plots
# %matplotlib notebook

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

And finally, create some convenience functions. 

In [ ]:
def download_file(url):
    """Download into the current working directory the
    file from Box given the direct URL
    
    Parameters
    ----------
    url : str
        URL to the file to be downloaded
        
    Returns
    -------
    download_filename : str
        Name of the downloaded file
    """
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise RuntimeError("Wrong URL - {}".format(url))
    download_filename = response.headers['Content-Disposition'].split('"')[1]
    with open(download_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    return download_filename

In [ ]:
def create_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None):
    ''' Function to generate a 2D image of the data, 
    with an option to highlight a specific pixel.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    plt.imshow(data_2d, origin='lower', cmap='gray', vmin=vmin, vmax=vmax)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)

    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    plt.colorbar(label='MJy/sr')

In [ ]:
def create_image_with_cat(data_2d, catalog, flux_limit, vmin, vmax, title=None):
    ''' Function to generate a 2D image of the data, 
    with sources overlaid.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    plt.imshow(data_2d, origin='lower', cmap='gray', vmin=vmin, vmax=vmax)
    
    for row in catalog:
        if row['aper_total_flux'] > flux_limit:
            plt.plot(row['xcentroid'], row['ycentroid'], marker='o', markersize='3', color='red')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    if title:
        plt.title(title)

    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    plt.colorbar(label='MJy/sr')

In [ ]:
def plot_spectra(spec, median_filter=None, title=None):
    ''' Function to generate the spectrum for a combined spectral product.
    '''
    
    fig = plt.figure(figsize=(10, 5))
    ax = plt.subplot()
    
    if median_filter:
        plt.plot(spec.spec_table['WAVELENGTH'], medfilt(spec.spec_table['FLUX'],median_filter)) 
        
    else: 
        plt.plot(spec.spec_table['WAVELENGTH'], spec.spec_table['FLUX']) 

    
    plt.xlabel('Wavelength (um)')
    plt.ylabel('Flux')
    
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)
    else:
        title='Final combined spectra from Stage 3'
        plt.title(title)

[Top of Page](#title_ID)

2.<font color='white'>-</font>Data in MAST <a class="anchor" id="mast"></a>
------------------

The JWST Data Management System (DMS) produces many products for each JWST observation, including the science files generated by the pipeline. The exact type and number of products depends on the instrument, its configuration, and observing mode. Observers should consult the [MAST documentation for information about standard data products](https://jwst-docs.stsci.edu/obtaining-data/data-discovery#DataDiscovery-Dataproducttypes). 

Of the many different data products produced by the calibration pipeline, most observers will find the science data files in MAST to be sufficient for their analysis. However, other data products such as guide star data, associations, and engineering data are also available. 

Standard science data files include:

* [uncalibrated raw data](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#uncalibrated-raw-data-uncal), identified by the suffix ```uncal```
* [countrate data](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#countrate-data-rate-and-rateints) produced by applying the Stage 1 (detector-level) corrections in order to compute count rates from the original accumulating signal ramps, identified by the suffix ```rate``` or ```rateints```
* [calibrated single exposures](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#calibrated-data-cal-and-calints), identified by the suffix ```cal```
* [resampled and/or combined exposures](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#resampled-2-d-data-i2d-and-s2d), identified by the suffixes ```i2d``` or ```s2d```
* [extracted spectroscopic 1D data](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#extracted-1-d-spectroscopic-data-x1d-and-x1dints), identified by the suffixes ```x1d``` or ```c1d```

In addition, there are also [several other products depending on the observing mode](https://jwst-pipeline.readthedocs.io/en/stable/jwst/data_products/science_products.html#source-catalog-cat), such as source and photometry catalogs, stacked PSF data, and NIRISS AMI derived data.  

[Top of Page](#title_ID)

3.<font color='white'>-</font>Example data for this exercise <a class="anchor" id="example"></a>
------------------

For this module, we will use calibrated NIRCam simulated imaging and wide field slitless spectroscopy (WFSS) exposures that are stored in Box. Let's grab the data:

In [ ]:
# For the catalog file:
catalog_file_link = 'https://stsci.box.com/shared/static/oicjg5rdiqdyh7r7oolzveodv5bxpv1a.ecsv'
output_catalog = download_file(catalog_file_link)

# For the FITS files:
fits_box_links = ['https://stsci.box.com/shared/static/ixfnu50ju78vs40dcec8i7w0u6kwtoli.fits', 'https://stsci.box.com/shared/static/6y3y6r6uoqwe90d5vhaaid8ntoiemicn.fits']
for boxfile in fits_box_links:
    fits_file = download_file(boxfile)
    
    
combined_i2d_file = "example_nircam_imaging_i2d.fits"
final_c1d_file = "example_nircam_wfss_c1d.fits"
output_catalog = "example_nircam_imaging_cat.ecsv"

[Top of Page](#title_ID)

4.<font color='white'>-</font>Data products: stage 3 (combined, rectified exposures)<a class="anchor" id="stage3"></a>
------------------

Stage 3 processing includes routines that work with multiple associated exposures to produce some kind of combined (mosaicked), rectified (aligned in a common output frame) product. There are unique pipeline modules for imaging, spectroscopic, coronagraphic, AMI, and TSO observations, and each produces specific outputs for the particular observing mode. The exposure level products are updated at this stage to provide the highest quality data products that include the results of ensemble processing (e.g., updated WCS, matching backgrounds, and a second pass at outlier detection). These products are available in MAST, along with the unrectified 2D and rectified 2D products. More information can be found in the [JWST User Documentation](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/algorithm-documentation/stages-of-processing). We also have a full list of data product types and the units of the data for each product [in the documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/product_types.html#data-product-types). 

We'll start by going through the various inputs and outputs for the mode-specific pipeline modules, and finish by revisiting our simulated data used in the previous notebooks. 

## 4.1.<font color='white'>-</font>Imaging<a class="anchor" id="s3-imaging"></a>

Stage 3 processing for direct imaging observations combines the calibrated data from multiple exposures (e.g., dithers or mosaics) into a single, rectified, distortion corrected product. Before being combined, the exposures receive additional corrections for astrometric alignment, background matching, and outlier detection. Coronagraphic imaging and time series imaging have their own separate pipeline modules. 

### A.<font color='white'>-</font>Input<a class="anchor" id="s3-imaging-input"></a>

The inputs to this stage are listed below.

* **2D calibrated images**
    * **Data model**: ImageModel
    * **File suffix**: ```_cal```
    * **Description**: The inputs to this pipeline are one or more Stage 2 calibrated (```_cal```) image products. An association file must be used as input in order to process and combine multiple images. If only a single ```_cal``` file is used as input, only the ```resample``` and ```source_catalog``` steps will be applied.
    

### B.<font color='white'>-</font>Output<a class="anchor" id="s3-imaging-output"></a>

The outputs of this stage are listed below.

* **CR-flagged exposures**
    * **Data model**: ImageModel
    * **File suffix**: ```_crf```
    * **Description**: If the ```outlier_detection``` step is applied, a new version of each input calibrated exposure is created with a data quality array that is updated to flag pixels detected as outliers. These files use the ```_crf``` (CR-Flagged) file suffix and include the association candidate ID as a new field in the original product root file name.
    
* **Resampled and combined 2D image**
    * **Data model**: DrizProductModel
    * **File suffix**: ```_i2d```
    * **Description**: The resampled 2D image that contains the combined, rectified association of exposures, which is the direct output of the ```resample``` step.  

* **Source catalog**
    * **Data model**: N/A
    * **File suffix**: ```_cat```
    * **Description**: The source catalog produced by the ```source_catalog``` step using the ```_i2d``` data product is saved as an ASCII file in ```ecsv``` format.

## 4.2.<font color='white'>-</font>Spectroscopy<a class="anchor" id="s3-spectroscopy"></a>

Stage 3 processing for spectroscopic observations combines the calibrated data from multiple exposures (e.g., dithers or nods) into a single combined 2D or 3D spectral data product and a combined 1D spectrum. Before being combined, exposures may receive additional corrections for background matching and subtraction, and outlier rejection. Time series data will go through the separate time series pipeline module. 

### A.<font color='white'>-</font>Input<a class="anchor" id="s3-spectroscopy-input"></a>

The inputs to this stage are listed below.

* **2D calibrated data**
    * **Data model**: ImageModel, IFUImageModel, SlitModel, or MultiSlitModel
    * **File suffix**: ```_cal```
    * **Description**: The inputs to this module should be in the form of an association file listing the exposures to be processed into combined data products. The individual exposures should be calibrated (```_cal```) products from Stage 2 processing. The member list for each product in the association file can also contain exposures of dedicated background targets, which are intended for use in the master_background step. These input exposures must be extracted 1D spectral products (```_x1d```) for the background target(s) produced by the Stage 2 pipeline. They must be listed in the association file with ```exptype``` values of ```background``` in order to be identified as background exposures.
   

### B.<font color='white'>-</font>Output<a class="anchor" id="s3-spectroscopy-output"></a>

The outputs of this stage are listed below.

* **Source-based calibrated data**
    * **Data model**: MultiExposureModel
    * **File suffix**: ```_cal```
    * **Description**: For NIRSpec fixed-slit, NIRSpec MOS, and NIRCam and NIRISS WFSS, which have a defined set of slits or sources, data from the input calibrated exposures is reorganized so that all instances of data for a particular source/slit are contained in a single product (referred to as “source-based” products), rather than input exposure-based products. Source-based collections of data are saved in intermediate files, one per source/slit. The root names of the source-based data products contain the source ID as an identifier and use the same ```_cal``` suffix as the input calibrated exposure files. The reorganized sets of data are sent to subsequent steps to process and combine all the data for one source at a time. 

* **CR-flagged exposures**
    * **Data model**: ImageModel
    * **File suffix**: ```_crf```
    * **Description**: If the ```outlier_detection``` step is applied, a new version of each input calibrated exposure is created with a data quality array that is updated to flag pixels detected as outliers. These files use the ```_crf``` (CR-Flagged) file suffix and include the association candidate ID as a new field in the original product root file name.
    
* **2D resampled and combined spectral data**
    * **Data model**: DrizProductModel
    * **File suffix**: ```_s2d```
    * **Description**: When processing non-IFU modes, a resampled/rectified 2D product is created containing the rectified and combined data for a given slit/source, which is the direct output of the ```resample``` step.     
    
* **3D resampled and combined spectral data**
    * **Data model**: IFUCubeModel
    * **File suffix**: ```_s3d```
    * **Description**: When processing IFU exposures, a resampled and combined 3D IFU cube product is created by the cube building step.

* **1D extracted spectral data**
    * **Data model**: MultiSpecModel
    * **File suffix**: ```_x1d```
    * **Description**: All inputs result in a 1D extracted spectral data product, saved as a ```_x1d``` file, which is normally the result of performing the 1D spectral extraction step on the combined ```_s2d``` or ```_s3d``` product. For NIRCam and NIRISS WFSS, and NIRISS SOSS, the 1D spectral extraction is performed on the individual unresampled 2D cutout images, which results in multiple 1D spectra per source in a ```_x1d``` product. Those spectra are combined using the subsequent 1D spectral combination step.
    
* **1D combined spectral data**
    * **Data model**: CombinedSpecModel
    * **File suffix**: ```_c1d```
    * **Description**: For NIRCam and NIRISS WFSS, and NIRISS SOSS, the 1D spectral combination step combines multiple 1D spectra for a given source into a final spectrum.     

## 4.3.<font color='white'>-</font>Aperture Masking Interferometry (AMI)<a class="anchor" id="s3-ami"></a>

Stage 3 processing for calibrated NIRISS AMI observations computes fringe parameters for individual exposures, averages the fringe results from multiple exposures, and, optionally, corrects science target fringe parameters using the fringe results from reference PSF targets.

### A.<font color='white'>-</font>Input<a class="anchor" id="s3-ami-input"></a>

The inputs to this stage are listed below.

* **2D calibrated images**
    * **Data model**: ImageModel
    * **File suffix**: ```_cal```
    * **Description**: Inputs need to be in the form of an association file that lists multiple science target exposures, and, optionally, reference PSF exposures. Individual exposures must be in the form of calibrated (```_cal```) data products from Stage 2 processing. 
 

### B.<font color='white'>-</font>Output<a class="anchor" id="s3-ami-output"></a>

The outputs of this stage are listed below.

* **Fringe parameter tables**
    * **Data model**: AmiLgModel
    * **File suffix**: ```_ami```
    * **Description**: For every input exposure, fringe parameters and closure phases caculated by the ```ami_analyze``` step are saved to a FITS table containing the fringe parameters and closure phases.

* (optional) **Averaged fringe parameters table**
    * **Data model**: AmiLgModel
    * **File suffix**: ```_amiavg``` or ```_psf-amiavg```
    * **Description**: If multiple target or reference PSF exposures are used as input and the ```–save_averages``` parameter is set to True, the ```ami_average``` step will save averaged results for the target in an ```_amiavg``` product and for the reference PSF in a ```_psf-amiavg``` product. 
   
* **Normalized fringe parameters table**
    * **Data model**: AmiLgModel
    * **File suffix**: ```_aminorm```
    * **Description**: If reference PSF exposures are included in the input association, the averaged AMI results for the target will be normalized by the averaged AMI results for the reference PSF and will be saved to an ```_aminorm``` product file. This file has the same FITS table format as the ```_ami``` products. 

## 4.4.<font color='white'>-</font>Coronagraphy<a class="anchor" id="s3-coronagraphy"></a>

Stage 3 coronagraphic processing is applied to associations of calibrated NIRCam coronagraphic and MIRI Lyot and 4QPM exposures, and is used to produce PSF-subtracted, resampled, combined images of the source object.

### A.<font color='white'>-</font>Input<a class="anchor" id="s3-coronagraphy-input"></a>

The inputs to this stage are listed below.

* **3D calibrated images**
    * **Data model**: CubeModel
    * **File suffix**: ```_calints```
    * **Description**: The input to this stage must be in the form of an association file that lists one or more exposures of a science target and one or more reference PSF targets. The individual target and reference PSF exposures should be in the form of 3D calibrated (```_calints```) data products from Stage 2 processing. Each pipeline step will loop over the 3D stack of per-integration images contained in each exposure. 
   

### B.<font color='white'>-</font>Output<a class="anchor" id="s3-coronagraphy-output"></a>

The outputs of this stage are listed below.

* **CR-flagged images**
    * **Data model**: CubeModel
    * **File suffix**: ```_crfints```
    * **Description**: If the ```outlier_detection``` step is applied, a new version of each input calibrated exposure is created with a data quality array that is updated to flag pixels detected as outliers. These files use the ```_crfints``` (CR-Flagged per integration) file suffix and include the association candidate ID as a new field in the original product root file name.

* **3D stacked PSF images**
    * **Data model**: CubeModel
    * **File suffix**: ```_psfstack```
    * **Description**: The data from each input PSF reference exposure are concatenated into a single combined 3D stack for use by subsequent steps. The stacked PSF data are written to a ```_psfstack``` product. 
    
* **4D aligned PSF images**
    * **Data model**: QuadModel
    * **File suffix**: ```_psfalign```
    * **Description**: For each science target exposure, all of the reference PSF images in the ```_psfstack``` product are aligned to each science target integration and saved to a 4D ```_psfalign``` product. 
        
* **3D PSF-subtracted images**
    * **Data model**: CubeModel
    * **File suffix**: ```_psfsub```
    * **Description**: For each science target exposure, the ```klip``` step applies PSF fitting and subtraction for each integration, resulting in a 3D stack of PSF-subtracted images. 
    
* **2D resampled image**
    * **Data model**: DrizProductModel
    * **File suffix**: ```_i2d```
    * **Description**: The ```resample``` step is applied to the CR-flagged products to create a single resampled and combined product for the science target. 

## 4.5.<font color='white'>-</font>Time Series Observation (TSO)<a class="anchor" id="s3-tso"></a>

Stage 3 TSO processing is applied to associations of calibrated TSO exposures (e.g. NIRCam TS imaging, NIRCam TS grism, NIRISS SOSS, NIRSpec BrightObj, MIRI LRS Slitless) and is used to produce calibrated time-series photometry or spectra of the source object.



### A.<font color='white'>-</font>Input<a class="anchor" id="s3-tso-input"></a>

The inputs to this stage are listed below.

* **3D calibrated images**
    * **Data model**: CubeModel
    * **File suffix**: ```_calints```
    * **Description**: The input is in the form of an association file listing multiple exposures or exposure segments of a science target. Individual inputs should be in the form of 3D calibrated (```_calints```) products from Stage 2 (either imaging or spectroscopic) processing. These products contain 3D stacks of per-integration images, and each pipeline step will loop over all of the integrations in each input. Many TSO exposures may contain a large number of integrations that make their individual exposure products too large (in terms of file size on disk) to be able to handle conveniently. In these cases, the uncalibrated raw data (```_uncal```) for a given exposure are split into multiple “segmented” products, each of which is identified with a segment number (see [segmented products](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/file_naming.html#segmented-files)). The input association file includes all ```_calints``` exposure segments. The ```outlier_detection``` step processes a single segment at a time, creating one output ```_crfints``` product per segment. The remaining steps will process each segment and concatenate the results into a single output product that contains results for all exposures and segments listed in the association.

### B.<font color='white'>-</font>Output<a class="anchor" id="s3-tso-output"></a>

The outputs of this stage are listed below.

* **CR-flagged images**
    * **Data model**: CubeModel
    * **File suffix**: ```_crfints```
    * **Description**: If the ```outlier_detection``` step is applied, a new version of each input calibrated exposure is created with a data quality array that is updated to flag pixels detected as outliers. These files use the ```_crfints``` (CR-Flagged per integration) file suffix and include the association candidate ID as a new field in the original product root file name.

* **Imaging photometry catalog**
    * **Data model**: N/A
    * **File suffix**: ```_phot```
    * **Description**: For imaging TS observations, a source catalog containing photometry results from all of the ```_crfints``` products is produced, organized as a function of integration time stamps.
       
* **1D extracted spectral data**
    * **Data model**: MultiSpecModel
    * **File suffix**: ```_x1dints```
    * **Description**: For spectroscopic TS observations, the 1D spectral extraction step is applied to all ```_crfints``` products to create a single ```_x1dints``` product containing 1D extracted spectral data for all integrations contained in the input exposures. 
       
* **Spectroscopic white-light catalog**
    * **Data model**: N/A
    * **File suffix**: ```_whtlt```
    * **Description**: For spectroscopic TS observations, the ```white_light``` step is applied to all of the 1D extracted spectral data in the ```_x1dints``` product to produce an ASCII catalog in ```ecsv``` format containing the wavelength-integrated white-light photometry of the source. The catalog lists the integrated white-light flux as a function of time, based on the integration time stamps.  

[Top of Page](#title_ID)

5.<font color='white'>-</font>Examining the products<a class="anchor" id="examine"></a>
------------------

Whew! That was a lot of information. Hopefully by now you're catching on to the pattern -- 

```uncal``` <span style="color:red">(raw)</span>

to

```rate``` or ```rateints``` <span style="color:brown">(detector corrections)</span>

to 

```cal``` or ```calints``` <span style="color:blue">(calibrated, unrectified individual products)</span>

to 

```i2d``` or ```x1d``` <span style="color:green">(combined, rectified products)</span>

with specific data models for the different formats, and some other data products (e.g., catalogs) sprinkled in for good measure. Let's take a look at some catalogs and some final combined, rectified prodcuts for imaging and spectroscopy in the following sections.

<div class="alert alert-block alert-warning">
<b>With so many data products, how do I know which model to use?!</b> 
    
When in doubt, you can always try to let the ```datamodels``` decide for you:

```python
with datamodels.open("myimage.fits") as model:
    ...
````
The ```datamodels.open()``` method checks if the ```DATAMODL``` FITS keyword has been set. If the keyword is not set, then ```datamodels.open()``` does its best to guess the best datamodel to use for you.
</div>

## 5.1.<font color='white'>-</font>Catalogs<a class="anchor" id="catalogs"></a>

Here, we'll focus on the catalog output from Stage 3 image processing, but other catalogs will have a similar ASCII format and file name (```.ecsv```). You can read more about the ```ecsv``` format [in the Astropy documentation here](https://docs.astropy.org/en/stable/io/ascii/write.html#ecsv-format). In short, it provides a convenient way to handle tables and associated metadata.

We can open the table using Astropy's ```Table```:

In [ ]:
# Load the catalog (use: image_catalog)
image_catalog = Table.read(output_catalog)

Working with astropy tables in notebooks is super convenient. There are a few additional options for displaying them:

```python
# Show the catalog in the Jupyter notebook
image_catalog.show_in_notebook()

# Show it in a separate browser
image_catalog.show_in_browser() 
```

This functionality isn't available in AWS, but you should be able to use it on your machine with local copies of notebooks and catalogs. 

In [ ]:
# What's it look like?
image_catalog

In [ ]:
# What's included in the information?
image_catalog.info

Print the columns and metadata:

In [ ]:
# Print the columns
image_catalog.colnames

In [ ]:
# Get the meta data
image_catalog.meta

Grab the data in one column: 

In [ ]:
# Grab the flux values (use: total_flux)
total_flux = image_catalog['aper_total_flux']

In [ ]:
# For our flux column, get the column name, unit, data type, and data
print('\nColumn description: ', total_flux.description)
print('Column name: ', total_flux.name)
print('Column units: ', total_flux.unit)
print('Column data type: ', total_flux.dtype)
print('Column data: ', total_flux.data)

RA, Dec values are [Skycoord objects](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html):

In [ ]:
# Show RA default
image_catalog['sky_centroid'].ra

In [ ]:
# Show Dec in deg
image_catalog['sky_centroid'].dec.deg

## 5.2.<font color='white'>-</font>Combined image<a class="anchor" id="comb-image"></a>

Here, we'll take a look at the final combined imaging data product from Stage 3, the ```_i2d``` file. Let's use the imaging simulation from our previous modules. 

In [ ]:
# Load the final combined 2D image (use: image)
image = datamodels.ImageModel(combined_i2d_file)

In [ ]:
# Check out the model structure
image.info()

In [ ]:
# Check out the meta data
image.meta.instance

In [ ]:
# What's the shape?
image.data.shape

In [ ]:
# Create an image of the final product
create_image(image.data, vmin=0, vmax=50, title="Final combined NIRCam image")

Let's overlay the catalog, as an example:

In [ ]:
# First, get a sigma-clipped mean of the flux to determine the scale for our image (use: clipped flux)
clipped_flux = sigma_clip(total_flux, sigma=2, maxiters=5, cenfunc=np.nanmean)
np.mean(clipped_flux)

In [ ]:
# Create our image with the catalog overlaid
create_image_with_cat(image.data, image_catalog, flux_limit=1e-5, vmin=0, vmax=50, title="Final image w/ catalog overlaid")

## 5.3.<font color='white'>-</font>Combined spectrum<a class="anchor" id="comb-spec"></a>

Here, we'll take a look at the final combined 1D spectrum from Stage 3 spectroscopic processing (```_c1d```. Let's use the WFSS simulation from our previous modules. 

In [ ]:
# Load the final combined 1D file (use: spectrum)
spectrum = datamodels.CombinedSpecModel(final_c1d_file)

In [ ]:
# Look at the model structure
spectrum.info()

In [ ]:
# Look at the metadata
spectrum.meta.instance

In [ ]:
# Show me the table values 
spectrum.spec_table

In [ ]:
# Plot it up!
plot_spectra(spectrum, median_filter=11)

[Top of Page](#title_ID)

6.<font color='white'>-</font>The end<a class="anchor" id="bye-bye"></a>
------------------

You made it! That's the end of our JWST data products JWebbinar. We hope you have a better idea of what to expect from your JWST observations, and feel more comfortable with the format and structure. As always, if you have any questions or concerns, you can find us through the [JWST Help Desk](https://stsci.service-now.com/jwst). 

[Top of Page](#title_ID)

7.<font color='white'>-</font>Exercise<a class="anchor" id="exercise"></a>
------------------

Now you try it!

In [ ]:
# How many sources are in the source catalog? 
len(image_catalog)

In [ ]:
# How many sources are identified by the pipeline as stars? 
np.sum(image_catalog['is_star']==False)

In [ ]:
# Where can you find information about the aperture corrections used for the catalog?
image_catalog.meta

In [ ]:
# Where do I find the spectral order for my combined spectrum?
spectrum.spectral_order

In [ ]:
# Where is the WCS information for the CombinedSpec model? 
spectrum.meta.wcs

In [ ]:
# What columns are in the spec_table? 
astrotab = Table(spectrum.spec_table).colnames
astrotab

[Top of Page](#title_ID)